In [3]:
from mlagents.envs import UnityEnvironment
import numpy as np 


In [4]:
train_mode = True  # Whether to run the environment in training or inference mode
#env_name = "Tennis_2_agents"
env_name = "Banana_2_agents"
env = UnityEnvironment(file_name=env_name, worker_id=1, seed=1)
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:mlagents.envs:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of Training Brains : 1
        Reset Parameters :
		
Unity brain name: BananaLearning
        Number of Visual Observations (per agent): 0
        Vector Observation space size (per agent): 53
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): [3, 3, 3, 2]
        Vector Action descriptions: , , , 


In [5]:
import random
import torch
import os
from collections import deque
import time
import matplotlib.pyplot as plt

#setting plotting options
%matplotlib inline
plt.style.use('ggplot')
np.set_printoptions(precision=3, linewidth=120)

#hiding matplotlib depreciate warnings
import warnings
warnings.filterwarnings("ignore")

#creating high resolution outputs
%config InlineBackend.figure_format='retina'





In [6]:
###path to save the models and collect tensorboard logs
model_dir = os.getcwd() + "/model_dir"


In [7]:
'''
###Instantiating the Unity Env and the Agent
env_info = env.reset(train_mode=True)[brain_name]
num_agents = len(env_info.agents)
ENV_ACTION_SIZE = len(brain.vector_action_space_size)
#ENV_ACTION_SIZE = len(brain.vector_action_space_size)
states = env_info.vector_observations
ENV_STATE_SIZE = states.shape[1]

print("num agents is: " + str(num_agents) +"  actions size: " + str(ENV_ACTION_SIZE) + " env state size is: " + str(ENV_STATE_SIZE))
'''

# Reset the environment    
env_info = env.reset(train_mode=True)[brain_name]     

# number of agents
num_agents = len(env_info.agents)

# size of each action
ENV_ACTION_SIZE = len(brain.vector_action_space_size)

# size of the state space 
states = env_info.vector_observations  # Array of states for all agents in teh enviroonments
ENV_STATE_SIZE = states.shape[1]

print('There are {} agents. Each observes a state with length: {} and act within an action space of length: {}'.format(states.shape[0], 
                                                                                                                       ENV_STATE_SIZE, 
                                                                                                                       ENV_ACTION_SIZE))


There are 2 agents. Each observes a state with length: 53 and act within an action space of length: 4


In [6]:
def random_agents():    
    for i_episode in range(1, 6):                              # play game for 5 episodes
        env_info = env.reset(train_mode=False)[brain_name]     # reset the environment
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        t=0                                                    # Initialize a counter for the nb of steps performed

        while True:
            t+=1
            actions = np.random.randn(num_agents, ENV_ACTION_SIZE) # select an action (for each agent)
            actions = np.clip(actions, -1, 1)                      # all actions between -1 and 1         
            env_info = env.step(actions)[brain_name]               # send all actions to tne environment
            next_states = env_info.vector_observations             # get next state (for each agent)
            rewards = env_info.rewards                             # get reward (for each agent)
            dones = env_info.local_done                            # see if episode finished

            scores += env_info.rewards                         # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                #print("   ** Debug: episode= {} steps={} rewards={} dones={}".format(i_episode, t,rewards,dones))
                break
        print('Score (max over agents) from episode {}: {}'.format(i_episode, np.max(scores)))

random_agents()

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.0


In [8]:
##MADDPG TRAINING OF AGENT
def plot_training(scores):
    # Plot the Score evolution during the training
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.tick_params(axis='x', colors='deepskyblue')
    ax.tick_params(axis='y', colors='deepskyblue')
    plt.plot(np.arange(1, len(scores)+1), scores, color='deepskyblue')
    plt.ylabel('Score', color='deepskyblue')
    plt.xlabel('Episode #', color='deepskyblue')
    plt.show()

In [9]:
from MADDPG import MADDPG
from hyperparams import *

def train():
    
    # Seeding
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    # Instantiate the MADDPG agents
    maddpg = MADDPG(ENV_STATE_SIZE, ENV_ACTION_SIZE, num_agents, SEED)

    # Monitor the score    
    scores_deque = deque(maxlen=100)
    all_scores = []
    all_avg_score = []
    
    
    # Intialize amplitude OU noise (will decay during training); done to ensure exploration
    noise = NOISE
    
    all_steps =0   
    
    # Training Loop
    for i_episode in range(NB_EPISODES+1):  
        env_info = env.reset(train_mode=True)[brain_name]          
        maddpg.reset()                                             
        states = env_info.vector_observations                      
        scores = np.zeros(num_agents)                              
        for steps in range(NB_STEPS):
            all_steps+=1
            actions = maddpg.act(states, noise)                    
            noise *= NOISE_REDUCTION                               
            env_info = env.step(actions)[brain_name]               
            next_states = env_info.vector_observations             
            rewards = env_info.rewards                             
            dones = env_info.local_done                            
            maddpg.step(states, actions, rewards, next_states, dones, i_episode)  
            scores += env_info.rewards                             
            #print("in episode score is: " + str(scores))
            states = next_states                                   
            if np.any(dones):                                      
                break
        
        # Save scores and compute average score over last 100 episodes       
        episode_score  = np.max(scores)  
        all_scores.append(episode_score)
        scores_deque.append(episode_score)
        avg_score = np.mean(scores_deque)
        print('\rEpisode {}\tAverage Score: {:.2f}\tEpisode score (max over agents): {:.2f}'.format(i_episode, avg_score, episode_score), end="")
        if i_episode>0 and i_episode % 50 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f} (nb of total steps={}   noise={:.4f})'.format(i_episode, avg_score, all_steps, noise))
            maddpg.checkpoints()
            all_avg_score.append(avg_score)
        # stop if objective is achieved
        if (i_episode > 99) and (avg_score >=0.5):
            print('\rEnvironment solved in {} episodes with an Average Score of {:.2f}'.format(i_episode, avg_score))
            maddpg.checkpoints()
            return all_scores

    return all_scores

In [10]:
scores = train()
plot_training(scores)

Episode 23	Average Score: 0.62	Episode score (max over agents): 1.000

KeyboardInterrupt: 